<a href="https://colab.research.google.com/github/bedadiggelmann/bina/blob/main/BINA%20Maria%20Haushalte%20Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

# Helper Functions



**Data Loading Functions**

This script loads data from a CSV file. Ensure that the path and the encoding match the data file specifics.

In [3]:
def load_data(filepath, encoding='ISO-8859-1', delimiter=';'):
    return pd.read_csv(filepath, encoding=encoding, delimiter=delimiter)

**Data Filtering Functions**

Filters the dataset for specific municipalities using a unique identifier contained in a column.

In [4]:
def filter_data_by_kanton_or_gemeinde(data, column_name, kanton_or_gemeinde_code):
    return data[data[column_name].str.contains(kanton_or_gemeinde_code)]

**Data Extraction Functions**

Extracts yearly data for given conditions and converts non-numeric entries appropriately.

In [5]:
def extract_yearly_data(gemeinde_data, condition_value, typ_value, start_col=4):
    yearly_data = gemeinde_data[
        (gemeinde_data["Anzahl/Anteil"] == condition_value) &
        (gemeinde_data["Leerwohnung (Typ)"].str.contains(typ_value, na=False))
    ].iloc[:, start_col:]
    yearly_data = yearly_data.replace('...', pd.NA)  # Replace '...' with NaN
    yearly_data = yearly_data.apply(pd.to_numeric, errors='coerce')  # Convert to numeric, coercing errors
    years = yearly_data.columns.tolist()  # Get the list of years (column names)
    return yearly_data.iloc[0] if not yearly_data.empty else pd.Series([pd.NA] * len(years), index=years)

# Gleiches auch für Anzahl Haushalte
def extract_yearly_data_2(gemeinde_data_2, start_col=2):
    yearly_data_2 = gemeinde_data_2
    yearly_data_2 = yearly_data_2.replace('...', pd.NA)  # Replace '...' with NaN
    yearly_data_2 = yearly_data_2.apply(pd.to_numeric, errors='coerce')  # Convert to numeric, coercing errors
    years_2 = yearly_data_2.columns.tolist()  # Get the list of years (column names)
    return yearly_data_2.iloc[0] if not yearly_data_2.empty else pd.Series([pd.NA] * len(years_2), index=years_2)

**Aggregate Data Functions**

In [6]:
# Helper function to map municipalities to Wahlkreis
def map_municipalities_to_wahlkreis(csv_data):
    wahlkreis_mapping = {
        # Example:
        # '3201': 'Wahlkreis Rorschach',
        # '3202': 'Wahlkreis St. Gallen',
    }
    # Update the dataset with a new column for Wahlkreis based on the mapping
    csv_data['Wahlkreis'] = csv_data['Grossregion (<<) / Kanton (-) / Gemeinde (......)'].map(wahlkreis_mapping)
    return csv_data

# Gleiches auch für Anzahl Haushalte
def aggregate_data_by_wahlkreis(csv_data, wahlkreis_name):
    # Filter data by Wahlkreis
    wahlkreis_data = csv_data[csv_data['Wahlkreis'] == wahlkreis_name]
    # Sum the data, replace 'Data_Column_Name' with the actual column name
    aggregated_data = wahlkreis_data.groupby('Wahlkreis').sum()
    return aggregated_data['Data_Column_Name']

**Plotting Functions**

Functions to plot time series and bar charts for visual comparisons.

In [7]:
def plot_line_chart(data_dict, title, xlabel, ylabel):
    fig, ax = plt.subplots(figsize=(14, 6))
    for label, data in data_dict.items():
        ax.plot(data.index, data.values, label=label, marker='o')
    ax.set_title(title)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.legend()
    ax.grid(True)
    plt.tight_layout()
    plt.show()


def plot_bar_chart(data_dict, title, xlabel, ylabel):
    fig, ax = plt.subplots(figsize=(14, 6))
    for label, data in data_dict.items():
        positions = range(len(data.index))
        ax.bar(positions, data.values, label=label)
    ax.set_title(title)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_ylim([0, 3])
    ax.set_xticks(range(len(data.index)))
    ax.set_xticklabels(data.index, rotation=45)
    ax.legend()
    ax.grid(True, which='both', linestyle='--', linewidth=0.5)
    plt.tight_layout()
    plt.show()

# Usage



**1. Load and Filter Data**

Load and filter data for each municipality and the canton.

In [10]:
csv_path = 'kanton_stgallen.csv'
csv_data = load_data(csv_path)

kanton_sg_data = filter_data_by_kanton_or_gemeinde(csv_data, 'Grossregion (<<) / Kanton (-) / Gemeinde (......)', 'Kanton St. Gallen')
gemeinde_haeggenschwil_data = filter_data_by_kanton_or_gemeinde(csv_data, 'Grossregion (<<) / Kanton (-) / Gemeinde (......)', '3201')
gemeinde_muolen_data = filter_data_by_kanton_or_gemeinde(csv_data, 'Grossregion (<<) / Kanton (-) / Gemeinde (......)', '3202')

#Gleiches auch für Anzahl Haushalte
csv_path_2 = 'Privathaushalte St. Gallen nach Jahr, Ort und Haushaltsgrösse.csv'
csv_data_2 = load_data(csv_path_2)

kanton_sg_data_2 = filter_data_by_kanton_or_gemeinde(csv_data_2, 'Kanton (-) / Bezirk (>>) / Gemeinde (......)', '=- St. Gallen')
gemeinde_haeggenschwil_data_2 = filter_data_by_kanton_or_gemeinde(csv_data_2, 'Kanton (-) / Bezirk (>>) / Gemeinde (......)', '3201')
gemeinde_muolen_data_2 = filter_data_by_kanton_or_gemeinde(csv_data_2, 'Kanton (-) / Bezirk (>>) / Gemeinde (......)', '3202')


**2. Extract Leerwohnungsanzahl and Leerwohnungsziffer for municipalities**


In [12]:
# Leerwohnungsanzahl
kanton_sg_leerwohnungsanzahl  = extract_yearly_data(
    kanton_sg_data,
    "Anzahl",
    "Leer stehende Wohnung - Total"
    )

haeggenschwil_leerwohnungsanzahl  = extract_yearly_data(
    gemeinde_haeggenschwil_data,
    "Anzahl",
    "Leer stehende Wohnung - Total"
    )

muolen_leerwohnungsanzahl = extract_yearly_data(
    gemeinde_muolen_data,
    "Anzahl",
    "Leer stehende Wohnung - Total"
    )

# Leerwohnungsziffer
kanton_sg_leerwohnungsziffer = extract_yearly_data(
    kanton_sg_data,
    'Leerwohnungsziffer',
    'Leer stehende Wohnung - Total'
)

haeggenschwil_leerwohnungsziffer = extract_yearly_data(
    gemeinde_haeggenschwil_data,
    'Leerwohnungsziffer',
    'Leer stehende Wohnung - Total'
)

muolen_leerwohnungsziffer = extract_yearly_data(
    gemeinde_muolen_data,
    'Leerwohnungsziffer',
    'Leer stehende Wohnung - Total'
)

#Gleiches auch für Anzahl Haushalte
kanton_sg_anz_haushalte  = extract_yearly_data_2(
    kanton_sg_data_2
    )

haeggenschwil_anz_haushalte = extract_yearly_data_2(
    gemeinde_haeggenschwil_data_2
    )

muolen_leerwohnungsanzahl = extract_yearly_data_2(
    gemeinde_muolen_data_2
    )




**3. Plot the Data**

In [5]:


# Anzahl der Leerwohnungen über die Jahre
plot_line_chart(
    {
        'Häggenschwil': haeggenschwil_leerwohnungsanzahl,
        'Muolen': muolen_leerwohnungsanzahl
    },
    'Anzahl der Leerwohnungen über die Jahre',
    'Jahr',
    'Anzahl der Leerwohnungen'
)

# Leerwohnungsziffer über die Jahre
plot_bar_chart(
    {
        'Kanton St. Gallen': kanton_sg_leerwohnungsziffer
    },
    'Leerwohnungsziffer über die Jahre im Kanton St. Gallen',
    'Jahr',
    'Leerwohnungsziffer (%)'
)

plot_line_chart(
    {
        'Häggenschwil': haeggenschwil_leerwohnungsziffer,
        'Muolen': muolen_leerwohnungsziffer
    },
    'Leerwohnungsziffer über die Jahre',
    'Jahr',
    'Leerwohnungsziffer (%)'
)


NameError: name 'plot_line_chart' is not defined